In [1]:
import pandas as pd
pdf19 = pd.read_csv('./data/pdf19.csv', encoding='euc-kr')
pdf19_input = pdf19.drop('학과', axis=1).to_numpy()
pdf19_target = pdf19['학과'].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(pdf19_input, pdf19_target, test_size=0.2, random_state=33)

----------------
<span style="color:yellow">6. 결정 트리 + 교차 검증, 그리드 서치, 랜덤 서치<span>

In [3]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=4, random_state=33)

In [34]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=12, shuffle=True, random_state=33)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
import numpy as np
np.mean(scores['test_score'])

0.6309523809523808

교차 검증만으로는 스코어가 개선되지 않았다.

In [78]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(3, 20, 1), 'min_samples_split': range(2, 100,10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=33), params, n_jobs=-1)
gs.fit(train_input, train_target)
np.max(gs.cv_results_['mean_test_score'])

0.6731494920174166

In [79]:
gs.best_params_

{'max_depth': 10,
 'min_impurity_decrease': 0.0006000000000000001,
 'min_samples_split': 2}

몇가지 파라미터를 변화시키며 그리드 서치를 하였더니 다소 개선된 결과가 나왔다.

In [80]:
from scipy.stats import uniform, randint
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth': randint(3, 20), 'min_samples_split': randint(2, 25), 'min_samples_leaf': randint(1,20)}

In [138]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=33), params, n_iter=40, n_jobs=-1, random_state=33)
gs.fit(train_input, train_target)
np.max(gs.cv_results_['mean_test_score'])

0.6803338171262701

In [139]:
gs.best_params_

{'max_depth': 14,
 'min_impurity_decrease': 0.0008552134671000478,
 'min_samples_leaf': 4,
 'min_samples_split': 15}

랜덤 서치 결과가 그리드 서치보다 약간 더 좋은 결과를 준다.  
여기까지의 훈련 결과로 얻은 모델을 최종 테스트 세트에 적용해 보자.

In [140]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.6666666666666666

훈련 결과와 비교해 볼 때 과대적합이 거의 사라졌다.

----------------
<span style="color:yellow">7. 랜덤 포레스트<span>

In [142]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=33)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9961949898442789 0.6614658925979681


교차 검증 훈련 결과는 상당한 과대적합이다.

In [143]:
rf.fit(train_input, train_target)
rf.feature_importances_

array([0.17062861, 0.05987805, 0.02100559, 0.04777507, 0.0451688 ,
       0.02659079, 0.01926088, 0.1089433 , 0.13228831, 0.11613129,
       0.12896729, 0.12336202])

단순 결정 트리에서 학년의 특성 중요도가 0.46447016 로 상당히 컸던 것에 비해 특성 중요도가 여러 특성에 골고루 분포된 느낌이다.

In [145]:
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=33)
rf.fit(train_input, train_target)
rf.oob_score_

0.6501901140684411

----------------
<span style="color:yellow">8. 엑스트라 트리<span>

In [146]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=33)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9961949898442789 0.6764150943396225


In [147]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20492286, 0.05902829, 0.02657742, 0.05343155, 0.04804219,
       0.02713094, 0.0222213 , 0.10308641, 0.11930729, 0.11403611,
       0.11157241, 0.11064323])

랜덤 포레스트와 비슷한 결과를 보여준다.

----------------
<span style="color:yellow">9. 그레디언트 부스팅<span>

In [158]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=33)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9419995486346198 0.6576197387518142


In [159]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.36379004, 0.07793725, 0.02917486, 0.06736518, 0.0594685 ,
       0.03952606, 0.03211523, 0.06844295, 0.10175149, 0.0483239 ,
       0.0746148 , 0.03748974])

학년의 특성 중요도가 다소 커진 것 외엔 위 결과와 크게 다르지 않다.

----------------
<span style="color:yellow">10. 히스토그램 기반 그레디언트 부스팅<span>

In [162]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=33)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9391469194312796 0.6309143686502178


In [167]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=33, n_jobs=-1)
result.importances_mean

array([0.3026616 , 0.15969582, 0.04068441, 0.04258555, 0.06958175,
       0.        , 0.        , 0.09391635, 0.12851711, 0.1121673 ,
       0.14334601, 0.09239544])

그레디언트 부스팅과 크게 다르지 않은 결과이다.  
이 모델을 최종 테스트 세트에 적용해 보자.

In [171]:
hgb.score(test_input, test_target)

0.7424242424242424

In [172]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=33, n_jobs=-1)
result.importances_mean

array([ 0.27727273,  0.03484848,  0.02575758,  0.03030303,  0.02575758,
       -0.0030303 ,  0.        ,  0.04545455,  0.02424242,  0.0530303 ,
        0.0969697 ,  0.02424242])

교차 검증 훈련 결과가 과대적합에 0.6309143686502178 이었던 것에 비하면 최종 테스트 결과가 의외로 기대보다 높게 나왔다.  
결정 트리 + 랜덤 서치의 최종 테스트 결과인 0.6666666666666666 보다도 향상된 결과이다.  
학년의 특성 중요도가 0.27727273 로 가장 높고 나머지에 비교적 골고루 분포된 느낌이다.

[결론             : 7_Couclusion.ipynb](./7_Couclusion.ipynb)